# Azure ML Reinforcement Learning Sample

Azure ML reinforcement learning is a managed service for running distributed RL (reinforcement learning) simulation and training using the Ray framework.

This example uses Ray rllib to train a Cartpole playing agent.

### Import libraries

In [ ]:
# Azure ML Core imports
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails

# Azure ML Reinforcement Learning imports
from azureml.contrib.train.rl import ReinforcementLearningEstimator, Ray
from azureml.contrib.train.rl import WorkerConfiguration

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

### Get Azure ML Workspace

In [ ]:
# Get workspace
ws = Workspace.from_config()

In [ ]:
# Get or create compute target
compute_target = ws.compute_targets['your-cpu-cluster']
compute_target

### Describe Cartpole Training Approach

This code is in the file `cartpole-one-cpu.py`.

```
import ray
import ray.tune as tune

if __name__ == "__main__":
   ray.init(redis_address = 'localhost:6379')

   tune.run("PPO",   
            config={
               "env": "CartPole-v0",
               "num_gpus": 0,
               "num_workers": 1
            },
            stop={
               "episode_reward_mean": 200,
               "time_total_s": 600
            },
            checkpoint_freq=1,
            checkpoint_at_end=True,
            local_dir='./logs'
            )

```

### Create Azure ML Experiment

In [ ]:
experiment_name='cartpole-one-cpu'

exp = Experiment(workspace=ws, name=experiment_name)

### Create Reinforcement Learning Estimator

The `ReinforcementLearningEstimator` is used to submit a job to Azure Machine Learning to start the Ray experiment run.

In [ ]:
estimator = ReinforcementLearningEstimator(

    # Location of source files
    source_directory='files',

    # Python script file
    entry_script='cartpole-one-cpu.py',
    
    # The Azure ML compute target set up for Ray head nodes
    compute_target=compute_target,
    
    # GPU usage
    use_gpu=False,
    
    # RL framework.  Currently must be Ray.
    rl_framework=Ray()
)

### Submit the estimator to start experiment

In [ ]:
run = exp.submit(config=estimator)

### Monitor experiment

Azure ML provides a Jupyter widget to show the real-time status of the experiment.

In [ ]:
RunDetails(run).show()